# Using SMO Cross-Platform
### This is a series of PowerShell SMO code to access SQL Server information.

1. First step - Install the SQL Server module using the "-AllowClobber":

``` Install-Module -Name SqlServer -AllowClobber ```

> _Note: Keep in mind! Latest version of SQL Server (2019+) will need more work with firewall and port configuration in Windows Systems._


2. Create list to SQLServers.

In [1]:
## - Create array list of Sql Server instances:
#$SQLServerInstanceName = @('sapien01,1449','mars,1433','sun,1445');
$SQLServerInstanceName = @('10.0.0.230,1449','10.0.0.227,1433','10.0.0.250,1433','sun,1445');
$SQLServerInstanceName;

The below script needs to be able to find the current output cell; this is an easy method to get it.

10.0.0.230,1449
10.0.0.227,1433
sun,1445


3. Execute foreach to collect SQLServer information.

In [2]:
## - Import SQL Server Module:
Import-Module SqlServer

## - Collect Server Edition Info:
$AllSqlServerEdition = $null;
[array]$AllSqlServerEdition = foreach($s in $SQLServerInstanceName){ 
	
	## - SQLServer Authentication:
	$SQLUserName = 'sa'; $SqlPwd = '$SqlPwd01!';
	
	## - Prepare connection to SQL Server:
	$SQLSrvConn = New-Object `
		Microsoft.SqlServer.Management.Common.SqlConnectionInfo($s, $SQLUserName, $SqlPwd);
	$SQLSrvObj = new-object Microsoft.SqlServer.Management.Smo.Server($SQLSrvConn);
	
	## - Sample T-SQL Queries:
	$SqlQuery = 'Select @@Version as FullVersion';
	
	## - Execute T-SQL Query:
	[array]$result = $SQLSrvObj.Databases['master'].ExecuteWithResults($SqlQuery);
	$GetVersion = $result.tables.Rows;
	
	## - SMO Get SQL Server Info:
	$SqlEdtion = $SQLSrvObj.Information `
	| Select-Object parent, platform, `
					@{ label = 'FullVersion'; Expression = { $GetVersion.FullVersion.Split(' - ')[0]; } }, `
					OSVersion, Edition, version, HostPlatform, HostDistribution;
					
	$SqlEdtion;
};

$AllSqlServerEdition | Format-List;



Parent           : [10.0.0.230,1449]
Platform         : NT x64
FullVersion      : Microsoft SQL Server 2017 (RTM-GDR) (KB4505224)
OSVersion        : 6.3 (19041)
Edition          : Developer Edition (64-bit)
Version          : 14.0.2027
HostPlatform     : Windows
HostDistribution : Windows 10 Pro

Parent           : [10.0.0.227,1433]
Platform         : NT x64
FullVersion      : Microsoft SQL Server 2019 (RTM-GDR) (KB4517790)
OSVersion        : 6.3 (20190)
Edition          : Developer Edition (64-bit)
Version          : 15.0.2070
HostPlatform     : Windows
HostDistribution : Windows 10 Pro

Parent           : [sun,1445]
Platform         : NT x64
FullVersion      : Microsoft SQL Server 2014 (SP3) (KB4022619)
OSVersion        : 6.3 (14393)
Edition          : Developer Edition (64-bit)
Version          : 12.0.6024
HostPlatform     : Windows
HostDistribution : 


